## DATA MART ASSIGNMENT

### Import Libraries

In [0]:
from functools import reduce
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, IntegerType, DateType, StringType, TimestampType, DateType
from pyspark.sql import *

### Configs and Constants

In [0]:
spark.conf.set("spark.sql.legacy.timeParserPolicy",'Legacy')

# File location and type
LIST_OF_TABLES = [ 
    "user",
    "order",
    "item",
    "event"
]

RAW_PREFIX = "_raw"
STAGE_PREFIX = "_stage"
S3_SOURCE_PREFIX = "s3://data-mart-source/"
S3_OUTPUT_PREFIX = "s3://data-mart-source/databricks_delta_lake/"
FILE_TYPE = "csv"
INFER_SCHEMA = "true"
FIRST_ROW_IS_HEADER = "true"
DELIMETER = ","

## Layer 1
▪ Contains external tables for all prerequisite files <br/>
▪ All attributes are of STRING type. No transformations are applied

In [0]:
# Iterating all tables.
for table_name in LIST_OF_TABLES:    
    file_location = f"{S3_SOURCE_PREFIX}{table_name}.{FILE_TYPE}"

    # The applied options are for CSV files. For other file types, these will be ignored.
    df = spark.read.format(FILE_TYPE) \
      .option("header", FIRST_ROW_IS_HEADER) \
      .option("sep", DELIMETER) \
      .option("quote", "\"") \
      .option("escape", "\"") \
      .load(file_location)
    
    # store the data in parquet format and creating external tables.
    df.write.mode("OVERWRITE").option("path", f"{S3_OUTPUT_PREFIX}{table_name}{RAW_PREFIX}").saveAsTable(f"{table_name}{RAW_PREFIX}")
    
    # unpersist the df from the memory
    df.unpersist()

## Layer 2 
▪ Contains all datasets from the first layer <br/>
▪ All attributes have common naming convention <br/>
▪ All attributes have proper datatypes based on the attribute name and common logic <br/>
▪ All struct collection attributes are flattened and transformed to proper data <br/>
▪ Fact tables are properly partitioned based on meaningful attributes

In [0]:
# All column names in snake case. (Following same convention for all attributes)
column_names = {
    "user":  [
                "created_at",
                "deleted_at",
                "email_address",
                "first_name",
                "id",
                "last_name",
                "merged_at",
                "parent_user_id"
            ],    
    "order": [
                "invoice_id",
                "line_item_id",
                "user_id",
                "item_id",
                "item_name",
                "item_category",
                "price",
                "created_at",
                "paid_at"
            ],
    "item": [
                "adjective",
                "category",
                "created_at",
                "id",
                "modifier",
                "name",
                "price"
            ],
    "event": [
                "event_id",
                "event_time",
                "user_id",
                "payload"
            ]
}

all_dataframes = {}

for table_name in LIST_OF_TABLES:
    df_table=spark.read.table(f"{table_name}{RAW_PREFIX}")
    old_columns = df_table.columns
    new_columns = column_names[table_name]
    stage_df = reduce(lambda df_table, idx: df_table.withColumnRenamed(old_columns[idx], new_columns[idx]), range(len(old_columns)), df_table)
    all_dataframes[table_name] = stage_df


#  All attributes have proper datatypes based on the attribute name and common logic
all_dataframes["user"] = all_dataframes["user"] \
    .withColumn('created_year',year(to_timestamp(col('created_at'), "yyyy-MM-dd HH:mm:ss"))) \
    .withColumn('created_at', to_timestamp(col('created_at'), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn('deleted_at', to_timestamp(col('deleted_at'), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn('id', col('id').cast(IntegerType())) \
    .withColumn('merged_at', to_timestamp(col('merged_at'), "yyyy-MM-dd HH:mm:ss"))

all_dataframes["order"]= all_dataframes["order"] \
    .withColumn('created_year',year(to_timestamp(col('created_at'), "MM/dd/yyyy HH:mm"))) \
    .withColumn('created_at', to_timestamp(col('created_at'), "MM/dd/yyyy HH:mm")) \
    .withColumn('paid_at', to_timestamp(col('paid_at'), "MM/dd/yyyy HH:mm")) \
    .withColumn('price', col('price').cast(IntegerType())) \
    .withColumn('item_id', col('item_id').cast(IntegerType())) \
    .withColumn('invoice_id', col('invoice_id').cast(IntegerType())) \
    .withColumn('user_id', col('user_id').cast(IntegerType())) \
    .withColumn('line_item_id', col('line_item_id').cast(IntegerType()))

all_dataframes["item"] = all_dataframes["item"] \
    .withColumn('created_year',year(to_timestamp(col('created_at'), "yyyy-MM-dd HH:mm:ss"))) \
    .withColumn('created_at', to_timestamp(col('created_at'), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn('id', col('id').cast(IntegerType())) \
    .withColumn('price', col('price').cast(DoubleType()))

all_dataframes["event"] = all_dataframes["event"] \
    .withColumn('created_year',year(to_timestamp(col('event_time'), "yyyy-MM-dd HH:mm:ss"))) \
    .withColumn('event_time', to_timestamp(col('event_time'), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn('user_id', col('user_id').cast(IntegerType()))


all_dataframes["event"] = all_dataframes["event"].select(col('event_id'),
                        col('created_year'),
                        col('event_time'),
                        col('user_id'),
                        get_json_object(col('`payload`'), '$.event_name').alias('event_name'),
                        get_json_object(col('`payload`'), '$.parameter_name').alias('parameter_name'),
                        get_json_object(col('`payload`'), '$.platform').alias('platform'),
                        get_json_object(col('`payload`'), '$.parameter_value').alias('parameter_value'))


In [0]:
# partitioning raw tables based on year 
for all_df in all_dataframes.keys():
    all_dataframes[all_df].write.option("path", f"{S3_OUTPUT_PREFIX}{all_df}{STAGE_PREFIX}").partitionBy("created_year").mode("OVERWRITE").saveAsTable(f"{all_df}{STAGE_PREFIX}")


d
##Top_Buyers

##### data mart with top 20 customers who contributed on the total sales the most with additional attributes:
o Total sales contributed <br/>
o Rank based on the total sales <br/>
o Last order creation date <br/>
o The overall most viewed item of a customer <br/>

In [0]:
#reading event_staging table

event_raw=spark.read.table('event_stage') \
              .select(col('event_id'),
                        col('event_time'),
                        col('user_id'),
                      col('parameter_name'),
                      col('parameter_value'))
                        
event_raw.limit(10).display()

In [0]:
#extracting all the items viewed most by any user
top_buyers_event=event_raw.select(col('user_id'),
                                 col('parameter_value'),
                                 col('parameter_name')) \
                          .filter(col('parameter_name')=='item_id') \
                          .groupBy(col('user_id'),col('parameter_name'),col('parameter_value')) \
                          .agg(count(col('parameter_value')).alias('total_number_of_times_item_viewed')) \
                          .withColumn('most_viewed_item',dense_rank().over(Window.partitionBy(col('user_id')).orderBy(col('total_number_of_times_item_viewed')))) \
                          .filter(col('most_viewed_item')==1) \
                          .select(col('user_id'),
                                 col('parameter_value').alias('item_id'))
top_buyers_event.limit(10).display()

In [0]:
# Aggregating data and calculating:
# 1) total orders placed by users.
# 2) total sales contributed by users.
# 3) last order date by that user.
# 4) rank of user based on total sales made.

top_buyers_raw=spark.read.table('order_stage') \
                    .select(col('user_id'),
                            col('invoice_id'),
                            col('price'),
                            col('created_at')) \
                    .groupBy(col('user_id')) \
                    .agg(count(col('invoice_id')).alias('total_orders_placed'),sum(col('price')).alias('total_sales_contributed'),max(to_date(col('created_at'),"MM/dd/yyyy HH:mm")).alias('last_order_created')) \
                    .withColumn('rank_of_user_based_on_total_sales',dense_rank().over(Window.orderBy(col('total_sales_contributed').desc()))) \
                    .filter(col('rank_of_user_based_on_total_sales')<=20)
top_buyers_raw.limit(10).display()

In [0]:
#appending most viewed item for that particular user
top_buyers=top_buyers_raw.join(top_buyers_event,['user_id'],how='left') \
                         .groupby('user_id','total_orders_placed','total_sales_contributed','rank_of_user_based_on_total_sales','last_order_created') \
                         .agg (collect_list('item_id').alias('most_viewed_item_id'))
top_buyers.limit(10).display()


## Top_Items

##### “top_items” data mart with all sold items with additional attributes:
o For every year (based on the created_at attribute): <br/>
-- Total number of an items sold in a particular year <br/>
-- Rank of an item based on the total number of items sold in a particular year <br/>
-- Total sales from an item in a particular year <br/>
-- Rank of an item based on the total sales in a particular year <br/>
o Total number of items sold in all years <br/>
o Rank of an item based on the total number of sales <br/>
o Total sales of an item in all years <br/>
o Rank of an item based on the total sales

### For a particular year
 takes *order_year* as parameter declared as a widget variable in the notebook

In [0]:
# Total number of items sold in a partuclar year
items_raw=spark.read.table('item_stage')
total_number_of_items_sold_in_year=items_raw.select(col('id').alias('item_id'),
                                                   year(to_timestamp(col('created_at'),"MM/dd/yyyy HH:mm")).alias('order_year')) \
                                            .groupBy(col('order_year')) \
                                            .agg(count(col('item_id')).alias('total_number_of_items_sold'))
total_number_of_items_sold_in_year.limit(10).display()

In [0]:
# Total sales from an item in a particular year
total_sales_of_items_sold_in_year=items_raw.select(col('price'),
                                                   year(to_timestamp(col('created_at'),"MM/dd/yyyy HH:mm")).alias('order_year')) \
                                            .groupBy(col('order_year')) \
                                            .agg(sum(col('price')).alias('total_sales_of_items_sold'))
total_sales_of_items_sold_in_year.limit(10).display()

In [0]:
# Rank of an item based on the total no of items sold in a particular year

rank_items_total_no_sold=items_raw.select(col('id').alias('item_id'),
                                                   year(to_timestamp(col('created_at'),"MM/dd/yyyy HH:mm")).alias('order_year')) \
                                            .groupBy(col('item_id'),col('order_year')) \
                                            .agg(count(col('item_id')).alias('total_number_of_items_sold')) \
                                            .withColumn('rank_of_an_item',dense_rank().over(Window.partitionBy('order_year').orderBy(col('total_number_of_items_sold').desc()))) 
rank_items_total_no_sold.limit(10).display()

In [0]:
# Rank of an item based on the total sales in a particular year

rank_item_total_sales_made=items_raw.select(col('id').alias('item_id'),
                                                   year(to_timestamp(col('created_at'),"MM/dd/yyyy HH:mm")).alias('order_year')) \
                                            .groupBy(col('item_id'),col('order_year')) \
                                            .agg(count(col('item_id')).alias('total_number_of_items_sold')) \
                                            .withColumn('rank_of_an_item',dense_rank().over(Window.partitionBy('order_year').orderBy(col('total_number_of_items_sold').desc()))) 
rank_item_total_sales_made.limit(10).display()

In [0]:
#Total number of items sold in all years

total_number_of_items_sold=items_raw.select(count(col('id').alias('item_id')).alias('total_number_of_items_sold_in_all_years'))
total_number_of_items_sold.limit(10).display()

In [0]:
#Total sales of an item in all years
total_sales_in_all_years=items_raw.select(col('price'),col('id').alias('item_id')) \
                                  .groupBy(col('item_id')) \
                                  .agg(sum(col('price')).alias('total_sales_of_items_sold'))
total_sales_in_all_years.limit(10).display()

In [0]:
# Rank of an item based on the total number of sales

item_rank_no_item_sold=items_raw.select(col('id').alias('item_id')) \
                                            .groupBy(col('item_id')) \
                                            .agg(count(col('item_id')).alias('total_number_of_items_sold')) \
                                            .withColumn('rank_of_an_item',dense_rank().over(Window.orderBy(col('total_number_of_items_sold').desc())))
item_rank_no_item_sold.limit(10).display()

In [0]:
#Rank of an item based on the total sales

item_rank_total_sales=items_raw.select(col('id').alias('item_id'),col('Price').alias('price')) \
                                            .groupBy(col('item_id')) \
                                            .agg(sum(col('price')).alias('total_sales_of_items_sold')) \
                                            .withColumn('rank_of_an_item',dense_rank().over(Window.orderBy(col('total_sales_of_items_sold').desc())))
item_rank_total_sales.limit(10).display()